In [70]:
import requests
import pandas as pd
from datetime import datetime

In [71]:
# API_KEY needs to saved as secret on GitHub
API_KEY = "XLC9cIQDZOgmCiMCT43D1umf6VeOFuGU"
base_url = "https://legislation.nysenate.gov/api/3/bills/"


In [72]:
def fetch_bills_from_api(session_year, committee_filter, page=1, limit=50):
    """
    Fetch bills from the API based on session year and committee filter.
    """
    url = f"{base_url}{session_year}?key={api_key}&page={page}&limit={limit}&committee={committee_filter}&sort=_score:desc,session:desc"
    
    try:
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            print(f"Response content: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [73]:
def flatten_json(nested_json, parent_key='', sep='_'):
    """
    Flatten a nested JSON into a flat dictionary.
    """
    items = []
    for k, v in nested_json.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_json(item, f"{new_key}_{i}", sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [74]:
def get_column_data(item, column_mappings):
    """
    Extract and return the required fields based on column mappings.
    """
    row_data = {}

    for column, key in column_mappings.items():
        if key:
            value = item.get(key, "")

            # Handle date formatting for 'Date Introed'
            if column == "Date Introed" and value:
                try:
                    # Convert to date format (YYYY-MM-DD)
                    formatted_date = datetime.fromisoformat(value).date()
                    row_data[column] = formatted_date
                except ValueError:
                    row_data[column] = value  # If it can't be converted, leave original value
            else:
                row_data[column] = value
        else:
            # Handle missing or empty columns
            if column == "Last Updated":
                row_data[column] = datetime.now().strftime("%Y-%m-%d")  # Set today's date
            else:
                row_data[column] = ""

    return row_data


In [75]:
def export_to_excel(bill_data, filename="NY_Senate_bills_data"):
    """
    Export the bill data into an Excel file.
    """
    flattened_data = []

    for item in bill_data:
        flattened_item = flatten_json(item)
        row_data = get_column_data(flattened_item, column_mappings)
        flattened_data.append(row_data)

    # Create a DataFrame and export it
    df = pd.DataFrame(flattened_data)

    if not df.empty:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{filename}_{timestamp}.xlsx"
        df.to_excel(filename, index=False)
        print(f"Data exported to {filename}")
    else:
        print("No data to export.")

# Column mappings for relevant fields
column_mappings = {
    "Last Updated": None,  # Today's date
    "Session": "session",
    "Chamber": "billType_desc",
    "Bill Number": "basePrintNo",
    "Date Introed": "publishedDateTime", 
    "Automated Status": "status_statusDesc",
    "Sponsor": "sponsor_member_fullName",
    "Co Sponsor Count": "amendments_items_coSponsors_size",
    "Co Sponsor Count Change": None,  
    "Bill Last Changed": "status_actionDate", 
    "Summary": "summary",
    "Committee": "status_committeeName",
    "Co Sponsors": None,  
    "Notes": None,  
    "Laws Impacted": "amendments_items_relatedLaws_items_AMEND_items",
    "Bill Link": None, 
    "Priority Level": None  
}

In [76]:
def fetch_and_export_bills(committee_filter, session_year, limit=50):
    """
    Fetch bills from the 2025 session and Elections Committee, then export to Excel.
    """
    all_bills = []
    page = 1
    while True:
        bill_data = fetch_bills_from_api(session_year, committee_filter, page, limit)

        if bill_data:
            # Extract the bills from the response
            bills = bill_data.get('result', {}).get('items', [])

            if bills:
                # Check each bill if it fulfills the committee filter and add it to all_bills
                for bill in bills:
                    if committee_filter in bill.get('status', {}).get('committeeName', ''):
                        all_bills.append(bill)

                # Check if there are more pages
                if len(bills) < limit:
                    break  # No more bills to fetch, end loop
                else:
                    page += 1  # Fetch the next page
            else:
                print("No bills found matching the filter.")
                break
        else:
            print("Error fetching bills or no data available.")
            break

    # If we have any valid bills, export them
    if all_bills:
        export_to_excel(all_bills)
    else:
        print("No bills to export.")

In [ ]:
def main():
    session_year = 2025  # 2025 session year
    committee_filter = "Elections"  # Filtering by Elections Committee

    # Fetch and export bills
    fetch_and_export_bills(committee_filter, session_year)

if __name__ == "__main__":
    main()